Halo connectivity of objects

This notebook is going to explore creating halo's around 'coral' objects that can be used to identify connectivity between corals through the flow. I think these halo's can be defined in the analysis part of the study, so all particleset output can be used

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import xarray as xr
from explfunctions import coraldistancemap
import cmocean

In [2]:
name = 'output-corals-regridded'
fb = 'forward' #variable to determine whether the flowfields are analysed 'forward' or 'backward' in time
runtime=timedelta(seconds=10)                            # total time to execute the particleset
outputdt = timedelta(seconds=0.01)
filename = name+'.nc'
dfilename = name+fb+str(runtime.seconds)+'.nc'

In [3]:
data = xr.open_dataset(dfilename)

flowdata = xr.open_dataset(filename)

In [4]:
umask = np.ma.masked_invalid(flowdata['U'][0])

In [5]:
cmask=np.copy(umask.mask)
for i in range(len(umask)):
    for j in range(len(umask[i])):
        if umask.mask[i,j] == True:
            if i>0 and i<len(umask)-1 and j>0 and j<len(umask[i])-1:
                if umask.mask[i-1,j]==True and umask.mask[i+1,j]==True and umask.mask[i,j-1]==True and umask.mask[i,j+1]==True:
                    cmask[i,j]=False
            elif i==0 and j>0 and j<len(umask[i])-1:
                if umask.mask[i+1,j]==True and umask.mask[i,j-1]==True and umask.mask[i,j+1]==True:
                    cmask[i,j]=False
            elif i==len(umask)-1 and j>0 and j<len(umask[i])-1:
                if umask.mask[i-1,j]==True and umask.mask[i,j-1]==True and umask.mask[i,j+1]==True:
                    cmask[i,j]=False
            elif i>0 and i<len(umask)-1 and j==0:
                if umask.mask[i-1,j]==True and umask.mask[i+1,j]==True and umask.mask[i,j+1]==True:
                    cmask[i,j]=False
            elif i>0 and i<len(umask)-1 and j==len(umask[i])-1:
                if umask.mask[i-1,j]==True and umask.mask[i+1,j]==True and umask.mask[i,j-1]==True:
                    cmask[i,j]=False
            elif i==0 and j==0:
                if umask.mask[i+1,j]==True and umask.mask[i,j+1]==True:
                    cmask[i,j]=False
            elif i==0 and j==len(umask[i])-1:
                if umask.mask[i+1,j]==True and umask.mask[i,j-1]==True:
                    cmask[i,j]=False
            elif i==len(umask)-1 and j==0:
                if umask.mask[i-1,j]==True and umask.mask[i,j+1]==True:
                    cmask[i,j]=False
            elif i==len(umask)-1 and j==len(umask[i])-1:
                if umask.mask[i-1,j]==True and umask.mask[i,j-1]==True:
                    cmask[i,j]=False
np.save('coastmask',cmask,allow_pickle=True)

In [6]:
xmesh,ymesh = np.meshgrid(flowdata['X'],flowdata['Y'])

In [7]:
# mat,coralmap = coraldistancemap(umask.mask,cmask,xmesh,ymesh)
# np.save('coralmap',coralmap,allow_pickle=True)
coralmap = np.load('coralmap.npy')

In [11]:
fig = plt.figure(figsize=(18,2))
ax = plt.axes()
ax.set_facecolor('k')
ax.contourf(flowdata['X'],flowdata['Y'],flowdata['U'][0],np.linspace(-10,-9,21),extend='both',cmap=cmocean.cm.ice)
sc = ax.scatter(xmesh[~umask.mask],ymesh[~umask.mask],c=coralmap[~umask.mask],s=10,cmap=cmocean.cm.amp_r)
cbar = fig.colorbar(sc,label='Distance from coral [m]')
plt.ylim(flowdata['Y'][-1],flowdata['Y'][0])
plt.xlim(flowdata['X'][0],flowdata['X'][-1])
plt.ylabel('Height [m]')
plt.xlabel('Horizontal distance [m]')

Text(0.5, 0, 'Horizontal distance [m]')

In [12]:
name = 'output-corals-regridded'
fb = 'forward' #variable to determine whether the flowfields are analysed 'forward' or 'backward' in time
runtime=timedelta(seconds=5)                            # total time to execute the particleset
outputdt = timedelta(seconds=0.1)
filename = name+'.nc'
dfilename = 'test'+name+fb+str(runtime.seconds)+'.nc'

In [13]:
distdata = xr.open_dataset(dfilename)

flowdata = xr.open_dataset(filename)

In [14]:
x,y = np.meshgrid(flowdata['X'],flowdata['Y'])
um = np.ma.masked_invalid(flowdata['U'][0,:,:])                    # retrieve mask from flowfield to take out points over coral objects

lons = np.ma.masked_array(x,mask=um.mask)             # mask points in meshgrid
lons = lons.flatten()
ds = np.ma.masked_array(y,mask=um.mask)                 # mask points in meshgrid
ds = ds.flatten()

usm = np.invert(um.mask)
lonm = np.ma.masked_array(x,mask=usm)             # mask points in meshgrid
lonm = lonm.flatten()
dm = np.ma.masked_array(y,mask=usm)                 # mask points in meshgrid
dm = dm.flatten()

In [15]:
%matplotlib qt
traj = 68000
fig = plt.figure(figsize=(18,2))
ax = plt.axes()
ax.set_facecolor('k')
# ax.scatter(lons,ds,s=7,c='c',label='flow gridpoint')
# ax.scatter(lonm,dm,s=15,c='k',label='coral gridpoint')
ax.contourf(flowdata['X'],flowdata['Y'],flowdata['U'][0],np.linspace(-10,-9,21),extend='both',cmap=cmocean.cm.ice)
sc = ax.scatter(distdata['lon'].isel(traj=traj,obs=slice(1,-1)),-distdata['z'].isel(traj=traj,obs=slice(1,-1)),c=distdata['d2c'].isel(traj=traj,obs=slice(1,-1)),marker='o',cmap=cmocean.cm.amp_r)
cbar = fig.colorbar(sc,label='Distance from coral [m]')
plt.ylabel('Height [m]')
plt.xlabel('Horizontal distance [m]')
plt.ylim(flowdata['Y'][-1],flowdata['Y'][0])
plt.xlim(flowdata['X'][0],flowdata['X'][-1])

(<xarray.DataArray 'X' ()>
 array(-0.5)
 Coordinates:
     X        float64 -0.5
 Attributes:
     units:    meters, <xarray.DataArray 'X' ()>
 array(8.49)
 Coordinates:
     X        float64 8.49
 Attributes:
     units:    meters)